# Analyse GitHub archives using GraphX

_Trying to detect open source communies based on contributions_

## Setup the environment to work with GraphX and Json data 

### Import some github data

In [ ]:
import sys.process._
if (!new java.io.File("/tmp/github.json").exists) {
  new java.net.URL("http://data.githubarchive.org/2015-01-01-15.json.gz")  #> new java.io.File("/tmp/github.json.gz") !!
  
  Seq("gunzip", "-f", "/tmp/github.json.gz")!!
}


### **The size of the data**

In [ ]:
:sh du -h /tmp/github.json

## First some Spark manipulation 

In [ ]:
val raw = sparkContext.textFile("/tmp/github.json")

### The number of lines in the file

In [ ]:
raw.count

### Convert line to JSON _(simple Map of Maps)_

In [ ]:
val json = raw.mapPartitions{ lines => 
  import com.fasterxml.jackson._
  import com.fasterxml.jackson.core._
  import com.fasterxml.jackson.databind._
  import com.fasterxml.jackson.module.scala._
  val mapper = new ObjectMapper()
  mapper.registerModule(DefaultScalaModule)
  lines.map(x => mapper.readValue(x, classOf[Map[String,Any]]))
}

### Let's look at the two first rows

In [ ]:
json.take(2).toList

## The graph part 

We could use the *actors* and the *repos* as vertices, and use the *event* as relationship between them.

There are *id*s for actor and repo, so we can directly use them in GraphX as such.

In [ ]:
import org.apache.spark.rdd._
import org.apache.spark.graphx._

### RDD vertices {Actors U Repos}

In [ ]:
val actors:RDD[(VertexId, (Short, String))] = json.map{ x => 
  val actor = x("actor").asInstanceOf[Map[String, Any]]
  val id = actor("id").toString.toLong
  val login = actor("login").toString
  (id, (0, login))
}
val repos:RDD[(VertexId, (Short, String))] = json.map{ x => 
  val repo = x("repo").asInstanceOf[Map[String, Any]]
  val id = repo("id").toString.toLong
  val name = repo("name").toString
  (id, (1, name))
}
val vertices:RDD[(VertexId, (Short, String))] = actors union repos

### RDD of Edges 

Now an **RDD** with the edges (including reverse ones, that is from repo to actor)

In [ ]:
// None → repo to actor
// Some("PushEvent") → actor pushed on repo
val edges:RDD[Edge[Option[String]]] = json.flatMap { x =>
  val event = x.get("type").map(_.toString)
  val actor = x("actor").asInstanceOf[Map[String, Any]]("id").toString.toLong
  val repo = x("repo").asInstanceOf[Map[String, Any]]("id").toString.toLong
  List(Edge(actor, repo, event), Edge(actor, repo, None))
}

### Graph

In [ ]:
val graph = Graph(vertices, edges)

## Open source working community 

A very very simple example of such extraction would simply be to extract the connected components 

So that, a component is the actors and repos having connections between them but not with other actor or repos. A connection being a collaboration.

### Computing connected components 

In [ ]:
val cc = graph.connectedComponents

The `cc` variable is the original graph but vertives' payload/properties is only the cluster to which is belongs. The cluster is characterized by the smallest `VertexId` in the cluster.

#### Number of connected components 

Computing the number of clusters can easily be done by counting the number of distinct `payload` for the vertices.

In [ ]:
<strong style="color: red">{cc.vertices.map(_._2).distinct.count}</strong>

### Clusters by language 

We can try to concentrate our analysis to specific languages, since we don't have the language information in the events data (we need extra call to the GitHub API for that) we'll take a naive approach, that is, **we'll only consider the repo having the language in their name** -- albeit it's not 100% safe.

#### Utility functions

The following function compute retrieves the cluster for a given cluster.

In [ ]:
import org.apache.spark.SparkContext._
def cluster(lgg:String) = {
  // collect all repos for the language `lgg`
  val lggRepos:List[(VertexId, (Short, String))] = vertices.filter { x => 
                    x._2._1 /*vertex type*/ == 1 /*repo*/ && 
                    x._2._2/*repo name*/.toLowerCase.contains(lgg) //here we SHOULD exclude the prefix of '/'
                }.collect().toList
  // keep only the set
  // ***** IN A CLUSTER →→→ THIS NEEDS TO BE A BROADCAST VARIABLE *****wwwwzeqc
  val lggRepoIds:List[Long] = lggRepos.map(_._1).distinct
  // clusters "id" for these repos → BROADCAST
  val clusterIds:List[Long] = cc.vertices.filter(x => lggRepoIds.contains(x._1))
                            .map(_._2)
                            .collect()
                            .toList
  // return the vertices being clustered sorted by decreasing cardinality
  val clusters:List[(Long, Iterable[Long])] = cc.vertices.filter{ x => clusterIds.contains(x._2) }
                 .groupBy(_._2)
                 .mapValues(_.map(_._1))
                 .collect().toList
                 .sortBy(_._2.size)
                 .reverse
  clusters
}

Shows the list of repos and actors included in the given cluster

In [ ]:
def showCluster(lgg:String, clusterIds:List[Long]) = {
  val c = graph.vertices
               .filter(x => clusterIds.contains(x._1))
               .collect().toList

  <div>
  <p><strong>Repos</strong></p>
  <ul>{ 
  c.collect { case (x, (1, r)) =>
           //show the repo
           val t = if (r.toLowerCase.contains(lgg)) <strong style="color: red;">{r}</strong> else r
             <li><a href={"http://github.com/"+r}>{t}</a></li> 
          }
  }</ul>
  <p><strong>Users</strong></p>
  <ul>{ 
  c.collect { case (x, (0, n)) =>
           //show the repo
             <li><a href={"http://github.com/"+n}>{n}</a></li> 
          }
  }</ul>
  </div>
}

## Javascript

In [ ]:
val js = cluster("js")

**Let's look at the 3 biggest clusters**

In [ ]:
layout(3, js.take(3).map(r => html(showCluster("js", r._2.toList))))

## Scala

In [ ]:
val scala = cluster("scala")

In [ ]:
layout(4, scala.map(r => html(showCluster("scala", r._2.toList))))


## Spark ^^ 

In [ ]:
val spark = cluster("spark")

In [ ]:
layout(4, spark.map(r => html(showCluster("spark", r._2.toList))))